In [ ]:
! pip install driftpy

In [ ]:
! pip install driftpy --upgrade

In [ ]:
# import sys
# sys.path.insert(0, 'driftpy/src/')
# import driftpy
# driftpy.__file__

In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

# LOAD devnet environment

In [2]:
import os
from driftpy.clearing_house import ClearingHouse
from driftpy.clearing_house_user import ClearingHouseUser

os.environ['ANCHOR_WALLET'] = os.path.expanduser('~/.config/solana/<YOUWALLET>.json')

drift_acct = await ClearingHouse.create_from_env('mainnet')
drift_user = ClearingHouseUser(drift_acct, drift_acct.program.provider.wallet.public_key)

In [3]:
pd.DataFrame((await drift_user.get_user_positions_account()).positions)

,market_index,base_asset_amount,quote_asset_amount,last_cumulative_funding_rate,last_cumulative_repeg_rebate,last_funding_rate_ts,stop_loss_price,stop_loss_amount,stop_profit_price,stop_profit_amount,transfer_to,padding0,padding1
0,0,-208242686073,2067087,280709713217662,0,1643727600,0,0,0,0,11111111111111111111111111111111,0,0
1,4,112413605268,749628,208775891305370,0,1643727606,0,0,0,0,11111111111111111111111111111111,0,0
2,3,-194244719980,995754,-106882990895767,0,1643727608,0,0,0,0,11111111111111111111111111111111,0,0
3,2,-3933989978,1000000,32205402480869760,0,1643727605,0,0,0,0,11111111111111111111111111111111,0,0
4,1,-263916013,1000000,394773480378601265,0,1643727615,0,0,0,0,11111111111111111111111111111111,0,0


In [4]:
market = await drift_user.clearing_house.get_market(0)
(await drift_user.get_position_value(0))/1e6

2.295601998653568

# load user state info

In [5]:
from typing import Optional, TypeVar, Type, cast
from driftpy.types import (
    PositionDirection,
    StateAccount,
    MarketsAccount,
    FundingPaymentHistoryAccount,
    FundingRateHistoryAccount,
    TradeHistoryAccount,
    LiquidationHistoryAccount,
    DepositHistoryAccount,
    ExtendedCurveHistoryAccount,
    User,
    UserPositions,
)
from driftpy.constants.markets import MARKETS
from driftpy.constants.numeric_constants import MARK_PRICE_PRECISION

#todo
QUOTE_PRECISION = 1e6
AMM_PRECISION = 1e13

In [12]:
drift_user_acct = await drift_user.get_user_account()
balance = (drift_user_acct.collateral/1e6)
balance

130034.75957

In [13]:
# load user positions
positions = cast(
                UserPositions,
                await drift_acct.program.account["UserPositions"].fetch(
                    drift_user_acct.positions
                ),
            )
positions_df = pd.DataFrame(positions.positions)[['market_index', 'base_asset_amount', 'quote_asset_amount']]
positions_df['base_asset_amount'] /= AMM_PRECISION
positions_df['quote_asset_amount'] /= QUOTE_PRECISION
positions_df = pd.DataFrame(MARKETS)[['symbol', 'market_index']].merge(positions_df)
positions_df.loc[positions_df.base_asset_amount==0,:] = np.nan
positions_df

,symbol,market_index,base_asset_amount,quote_asset_amount
0,SOL-PERP,0.0,-0.020824,2.067087
1,BTC-PERP,1.0,-0.000026,1.000000
2,ETH-PERP,2.0,-0.000393,1.000000
3,LUNA-PERP,3.0,-0.019424,0.995754
4,AVAX-PERP,4.0,0.011241,0.749628


# load predicted funding

In [14]:
# !pip install pythclient
# import asyncio

from pythclient.pythaccounts import PythPriceAccount
from pythclient.solana import (SolanaClient, SolanaPublicKey, SOLANA_DEVNET_HTTP_ENDPOINT, SOLANA_DEVNET_WS_ENDPOINT,
SOLANA_MAINNET_HTTP_ENDPOINT, SOLANA_MAINNET_HTTP_ENDPOINT)

async def get_pyth_price(address):
    # devnet DOGE/USD price account key (available on pyth.network website)
    account_key = SolanaPublicKey(address)
    solana_client = SolanaClient(endpoint=SOLANA_MAINNET_HTTP_ENDPOINT, ws_endpoint=SOLANA_MAINNET_HTTP_ENDPOINT)
    price: PythPriceAccount = PythPriceAccount(account_key, solana_client)

    await price.update()
    # print(dir(price.derivations))
    twap_result = (price.derivations.get('TWAPVALUE'), price.derivations.get('TWACVALUE'))
    price_result = (price.aggregate_price, price.aggregate_price_confidence_interval)
    print(twap_result)
    return price
    # # print(price.aggregate_price, "±", price.aggregate_price_confidence_interval)
    # return result

In [15]:
from datetime import datetime, timedelta

markets = await drift_acct.get_markets_account()


def calculate_capped_funding_rate(markets_summary):
    next_funding = (markets_summary['last_mark_price_twap'] \
                         - markets_summary['last_oracle_price_twap'])/24
    fd1 =  next_funding/MARK_PRICE_PRECISION \
    * (markets_summary['base_asset_amount']/AMM_PRECISION)
    fd2_m = (((markets_summary['total_fee_minus_distributions']
                             - markets_summary['total_fee']/2)*.66666)/QUOTE_PRECISION)
    
    fd2_u =  next_funding/MARK_PRICE_PRECISION \
    * (markets_summary['base_asset_amount_long'])
    
    
    est_fee_pool_funding_revenue = markets_summary["base_asset_amount"]/AMM_PRECISION * next_funding/MARK_PRICE_PRECISION

    drift_capped_fund_rate = (markets_summary[["base_asset_amount_long", "base_asset_amount_short"]].abs().min(axis=1)  \
                       * next_funding + fd2_m*MARK_PRICE_PRECISION*AMM_PRECISION) \
    / markets_summary[["base_asset_amount_long", "base_asset_amount_short"]].abs().max(axis=1)

    capped_funding = drift_capped_fund_rate/markets_summary['last_oracle_price_twap'] * 100
    capped_funding[fd2_m+est_fee_pool_funding_revenue>0] = np.nan
    
    return capped_funding
    

async def calculate_market_summary(markets):
    FUNDING_PRECISION = 1e4
    
    markets_summary = pd.concat([
        pd.DataFrame(MARKETS).iloc[:,:3],
    pd.DataFrame(markets.markets),
    pd.DataFrame([x.amm for x in markets.markets]),           
              ],axis=1).dropna(subset=['symbol'])

    last_funding_ts = pd.to_datetime(markets.markets[0].amm.last_funding_rate_ts*1e9)
    next_funding_ts = last_funding_ts + timedelta(hours=1)
    next_funding_ts

    summary = {}
    
    next_funding = (markets_summary['last_mark_price_twap'] \
                         - markets_summary['last_oracle_price_twap'])/24
    summary['next_funding_rate(%)'] = next_funding\
        /markets_summary['last_oracle_price_twap'] * 100
    
    
    summary['next_funding_rate_capped(%)']  = calculate_capped_funding_rate(markets_summary)
    
    summary['next_funding_rate(%APR)'] = (summary['next_funding_rate(%)'] * 24 * 365.25).round(2)
    
    # next_est_capped_funding_revenue = \
    # markets_summary[["base_asset_amount_long", "base_asset_amount_short"]].abs().min(axis=1) * next_funding \
    # - markets_summary[["base_asset_amount_long", "base_asset_amount_short"]].abs().max(axis=1) * drift_capped_fund_rate
    
    summary['mark_price'] = (markets_summary['quote_asset_reserve'] \
                         /markets_summary['base_asset_reserve'])\
    *markets_summary['peg_multiplier']/1e3
    
    prices = []
    twaps = []
    confs = []
    twacs = []
    for x in markets_summary['oracle'].values.tolist():
        price = await get_pyth_price(str(x))
        prices.append(price.aggregate_price)
        confs.append(price.aggregate_price_confidence_interval)        
    summary['oracle_price'] = prices
    summary['oracle_conf'] = confs
    
    df = pd.concat([pd.DataFrame(MARKETS).iloc[:,:3], pd.DataFrame(summary)],axis=1)
    
    # df.loc[fd2_m+est_fee_pool_funding_revenue>0,
    #      ['next_funding_rate_capped(%)']] = np.nan
    
    return df

In [16]:
import warnings
warnings.filterwarnings('ignore')
market_summary = await calculate_market_summary(markets)

(None, None)


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x130cd8610>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13127d000>, 109178.0420485)]']
connector: <aiohttp.connector.TCPConnector object at 0x130cd9bd0>


(None, None)


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x130cd9e40>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13127c220>, 109178.521105791)]']
connector: <aiohttp.connector.TCPConnector object at 0x130c83c10>


(None, None)


...

(None, None)


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x130cd9c60>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13127e1a0>, 109181.026611666)]']
connector: <aiohttp.connector.TCPConnector object at 0x130cd9ea0>


(None, None)


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x130d0f1c0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13127d1e0>, 109181.351755083)]']
connector: <aiohttp.connector.TCPConnector object at 0x130d0f340>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x130d0fc10>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x13127d000>, 109181.682381625)]']
connector: <aiohttp.connector.TCPConnector object at 0x130d0f460>


(None, None)


In [17]:
# oracle_mark_spread = market_summary['mark_price']-market_summary['oracle_price']

# funding_whatif = np.sign(oracle_mark_spread)*abs(oracle_mark_spread - market_summary['oracle_conf'])\
# /market_summary['oracle_price']
# pd.concat([funding_whatif, market_summary['next_funding_rate']],axis=1)
# (funding_whatif - market_summary['next_funding_rate'])#/market_summary['next_funding_rate']

In [18]:
market_summary

,symbol,base_asset_symbol,market_index,next_funding_rate(%),next_funding_rate_capped(%),next_funding_rate(%APR),mark_price,oracle_price,oracle_conf
0,SOL-PERP,SOL,0,-0.000303,NaN,-2.66,110.236871,110.167708,0.122662
1,BTC-PERP,BTC,1,0.001382,NaN,12.11,40721.80424,40669.473000,17.957000
2,ETH-PERP,ETH,2,-0.002189,NaN,-19.19,2956.05543,2957.478500,0.417750
3,LUNA-PERP,LUNA,3,0.003208,NaN,28.12,52.731732,52.669854,0.054910
4,AVAX-PERP,AVAX,4,-0.001286,NaN,-11.27,75.981069,76.005171,0.090219
5,BNB-PERP,BNB,5,-0.002116,NaN,-18.55,395.960193,396.753730,0.288035
6,MATIC-PERP,MATIC,6,-0.001635,NaN,-14.33,1.671137,1.671870,0.001694
7,ATOM-PERP,ATOM,7,0.000357,NaN,3.13,30.726488,30.704276,0.014644
8,DOT-PERP,DOT,8,-0.001058,NaN,-9.28,20.103447,20.146164,0.018725
9,ADA-PERP,ADA,9,-0.014087,NaN,-123.49,1.113862,1.119950,0.000785
